In [662]:
import numpy as np

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [663]:
def householder(x):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    x = np.asarray(x, dtype=float)
    y = np.zeros(len(x))
    y[0] = np.linalg.norm(x)
    if x.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % x.ndim)
    elif abs((y[0] - x[0]) / y[0]) < 1e-10 and x[0] > 0:  #если близки друг к другу 
        u = np.asarray(x, dtype=float) 
        u[0] = (x[0]**2 - y[0]**2) / (x[0] + y[0]) 
        u = u / np.linalg.norm(u)
    else:
        u = (x-y) / np.linalg.norm(x-y)
    H = np.eye(len(x))-2*np.outer(u,u)
    return y, H

Test your function using tests below:

In [664]:
# Test I.1 (10% of the total grade).

v = np.array([1,2,3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v, atol=1e-10)
assert_allclose(np.dot(h, v), v1, atol=1e-10)

In [665]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)
assert_allclose(np.dot(h, v1), vec, atol=1e-10)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [666]:
def qr_decomp(A):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    m, n = A.shape
    R = np.array(A, copy=True, dtype=float)
    x = A[0:,0]
    y, H = householder(x)
    R = H @ R
    Q = H.T
    for i in range(1, min(m-1,n)):
        Qi = np.eye(m)
        Ri = R[i:,i:]
        x = Ri[0:, 0]
        y, H = householder(x)
        Qi[i:,i:] = H
        Ri = H @ Ri
        Q = Q @ Qi.T
        R[i:,i:] = Ri
    return R, Q

Мой тест

In [667]:
A = np.array([[-1, 2, 3],[1, 5, -3],[1,2, -7],[1, 6, -3]])
print('A= ', A)
print("R = ","\n",qr_decomp(A)[0],"\n","Q = ","\n", qr_decomp(A)[1], "\n","QR = ","\n", qr_decomp(A)[1]@qr_decomp(A)[0])

A=  [[-1  2  3]
 [ 1  5 -3]
 [ 1  2 -7]
 [ 1  6 -3]]
R =  
 [[ 2.          5.5        -8.        ]
 [-0.          6.2249498   0.4819316 ]
 [-0.         -0.          3.43041425]
 [-0.          0.         -0.        ]] 
 Q =  
 [[-0.5         0.76305836 -0.39871043  0.09365858]
 [ 0.5         0.3614487   0.24073082  0.74926865]
 [ 0.5        -0.1204829  -0.85760356  0.        ]
 [ 0.5         0.52209256  0.21816231 -0.65561007]] 
 QR =  
 [[-1.  2.  3.]
 [ 1.  5. -3.]
 [ 1.  2. -7.]
 [ 1.  6. -3.]]


Видно, что разложение работает

In [668]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [669]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
r, q = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [670]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)
print("q = ","\n",q,"\n","qq = ","\n", qq,"\n", "r = ","\n", r,"\n", "rr = ","\n", rr,"\n", "aa = ","\n",qq @ rr,"\n","a = ","\n",q @ r)

q =  
 [[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]] 
 qq =  
 [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]] 
 r =  
 [[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.          0.5496046 ]
 [ 0.          0.         -0.        ]
 [ 0.          0.          0.        ]] 
 rr =  
 [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.          0.  

*Enter your explanation here* (10% of the total grade, peer-graded)

Видно, что q, r и qq, rr не совпадают по
а) двум последним столбцам для q и qq, что связано с тем, что две последних строки у r и rr равны нулю следовательно здесь проявляется многозначность
б) знакам некоторых элементов, что может быть объяснено выбором базисного вектора, по которому происходит разложение

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

Функции работает аналогично part II (идея та же самая)

In [671]:
def householder_u(x): #создает вектор 
    x = np.asarray(x, dtype=float)
    if x.ndim != 1:
        raise ValueError("x.ndim = %s, expected 1" % x.ndim)
    else:
        y = np.zeros(len(x))
        y[0] = np.linalg.norm(x)
        u = (x-y) / np.linalg.norm(x-y)
        u = np.reshape(u, (len(u),1)) #именно столбец
        return y, u

In [682]:
def qr_decomp_u(A): #первая функция 
    m, n = A.shape
    U=np.eye(m)
    R = np.array(A, copy=True, dtype=float)
    Ru = np.array(A, copy=True, dtype=float)
    x = A[0:,0]
    y, u = householder_u(x)
    uT=np.reshape(u, (1,len(u)))
    Ru = u.T @ Ru
    Ru = 2*u @ Ru
    R = R - Ru #умножение на матрицу H
    U[0:,0] = uT #хранит векторы u
    for i in range(1,min(m-1,n)):
        Ri = R[i:,i:]
        Riu = R[i:,i:]
        x = Ri[0:, 0]
        y, u = householder_u(x)
        uT=np.reshape(u, (1,len(u))) #строка
        Riu = uT @ Riu
        Riu = 2*u @ Riu
        Ri = Ri - Riu
        U[i:,i]=uT 
        R[i:,i:] = Ri
    return R, U

In [683]:
def qr_decomp_q(U,A): #вторая функция
    m, n = A.shape
    R = np.array(A, copy=True, dtype=float)
    Ru = np.array(A, copy=True, dtype=float)
    uT=U[0:,0]
    u=np.reshape(uT, (len(uT),1))
    Ru = u.T @ Ru
    Ru = 2*u @ Ru
    R = R - Ru #умножение на H 
    U[0:,0] = uT
    for i in range(1,min(m-1,n)): #запускаем аналогичного рода итерации
        Ri = R[i:,i:]
        Riu = R[i:,i:]
        uT=U[i:,i]   #вытаскиваю u
        u=np.reshape(uT, (len(uT),1))  #столбец
        uT=np.reshape(u, (1,len(uT))) #строка
        Riu = uT @ Riu
        Riu = 2*u @ Riu
        Ri = Ri - Riu
        U[i:,i]=uT
        R[i:,i:] = Ri
    return R

Тесты

In [684]:
A = np.array([[-1, 2, 3],[1, 5, -3],[4, 2, 3],[7, 2, 3]])
print('A= ', A)
print("R = ","\n",qr_decomp_u(A)[0],"\n","U  = ","\n",qr_decomp_u(A)[1],"\n","U содержит в себе набор векторов u 1-предпоследжний столбцы начало которых соответсвуют диагональным элементам (без последнего)","\n")

A=  [[-1  2  3]
 [ 1  5 -3]
 [ 4  2  3]
 [ 7  2  3]]
R =  
 [[ 8.18535277  3.05423611  3.298575  ]
 [ 0.          5.26038419 -1.34488787]
 [ 0.          0.          4.82811346]
 [ 0.          0.          0.        ]] 
 U  =  
 [[-0.74905589  0.          0.          0.        ]
 [ 0.08154895 -0.18883533  0.          0.        ]
 [ 0.32619581  0.7756134  -0.95949279  0.        ]
 [ 0.57084266  0.60229982 -0.2817332   1.        ]] 
 U содержит в себе набор векторов u 1-предпоследжний столбцы начало которых соответсвуют диагональным элементам (без последнего) 



In [675]:
R, U = qr_decomp_u(A)
print("A = ","\n",A)
print("U = ","\n",U)
print("R из первой функции = ","\n",R)
print("R из второй функции = ","\n",qr_decomp_q(U, A))

A =  
 [[-1  2  3]
 [ 1  5 -3]
 [ 4  2  3]
 [ 7  2  3]]
U =  
 [[-0.74905589  0.          0.          0.        ]
 [ 0.08154895 -0.18883533  0.          0.        ]
 [ 0.32619581  0.7756134  -0.95949279  0.        ]
 [ 0.57084266  0.60229982 -0.2817332   1.        ]]
R из первой функции =  
 [[ 8.18535277  3.05423611  3.298575  ]
 [ 0.          5.26038419 -1.34488787]
 [ 0.          0.          4.82811346]
 [ 0.          0.          0.        ]]
R из второй функции =  
 [[ 8.18535277  3.05423611  3.298575  ]
 [ 0.          5.26038419 -1.34488787]
 [ 0.          0.          4.82811346]
 [ 0.          0.          0.        ]]


In [676]:
A = np.array([[-1, 2, 3],[1, 5, -3],[-6, 7, -2]])
print('A= ', A)
print("R = ","\n",qr_decomp_u(A)[0],"\n","U  = ","\n",qr_decomp_u(A)[1],"\n","U содержит в себе набор векторов u 1-предпоследжний столбцы начало которых соответсвуют диагональным элементам (без последнего)","\n")

A=  [[-1  2  3]
 [ 1  5 -3]
 [-6  7 -2]]
R =  
 [[ 6.164414   -6.32663542  0.97332853]
 [-0.          6.16227914 -2.73309678]
 [ 0.         -0.          3.68548689]] 
 U  =  
 [[-0.76230618  0.          0.        ]
 [ 0.10640175 -0.0021641   0.        ]
 [-0.63841049  0.99999766  1.        ]] 
 U содержит в себе набор векторов u 1-предпоследжний столбцы начало которых соответсвуют диагональным элементам (без последнего) 



In [677]:
R, U = qr_decomp_u(A)
print("A = ","\n",A)
print("U = ","\n",U)
print("R из первой функции = ","\n",R)
print("R из второй функции = ","\n",qr_decomp_q(U, A))

A =  
 [[-1  2  3]
 [ 1  5 -3]
 [-6  7 -2]]
U =  
 [[-0.76230618  0.          0.        ]
 [ 0.10640175 -0.0021641   0.        ]
 [-0.63841049  0.99999766  1.        ]]
R из первой функции =  
 [[ 6.164414   -6.32663542  0.97332853]
 [-0.          6.16227914 -2.73309678]
 [ 0.         -0.          3.68548689]]
R из второй функции =  
 [[ 6.164414   -6.32663542  0.97332853]
 [-0.          6.16227914 -2.73309678]
 [ 0.         -0.          3.68548689]]


In [678]:
A = np.array([[-1, 2, 3],[1, 5, -3]])
print('A= ', A)
print("R = ","\n",qr_decomp_u(A)[0],"\n","U  = ","\n",qr_decomp_u(A)[1],"\n","U содержит в себе набор векторов u 1-предпоследжний столбцы начало которых соответсвуют диагональным элементам (без последнего)","\n")

A=  [[-1  2  3]
 [ 1  5 -3]]
R =  
 [[ 1.41421356  2.12132034 -4.24264069]
 [ 0.          4.94974747  0.        ]] 
 U  =  
 [[-0.92387953  0.        ]
 [ 0.38268343  1.        ]] 
 U содержит в себе набор векторов u 1-предпоследжний столбцы начало которых соответсвуют диагональным элементам (без последнего) 



In [679]:
R, U = qr_decomp_u(A)
print("A = ","\n",A)
print("U = ","\n",U)
print("R из первой функции = ","\n",R)
print("R из второй функции = ","\n",qr_decomp_q(U, A))

A =  
 [[-1  2  3]
 [ 1  5 -3]]
U =  
 [[-0.92387953  0.        ]
 [ 0.38268343  1.        ]]
R из первой функции =  
 [[ 1.41421356  2.12132034 -4.24264069]
 [ 0.          4.94974747  0.        ]]
R из второй функции =  
 [[ 1.41421356  2.12132034 -4.24264069]
 [ 0.          4.94974747  0.        ]]


Видно что функции выдают одинаковые R